In [5]:
# importing the csv files 
import pandas as pd

df_amazon = pd.read_csv('amazon_data.csv')
df_coke = pd.read_csv('coke_data.csv')
df_jnj = pd.read_csv('j&j_data.csv')

In [ ]:
# preprocessing the date column (TODO:esther)